### First exploration

This file includes an initial exploration of the dike model from the final assignment.
With this, it is seen how the model is used and the code for assignment 1-3 was implemented.

In [1]:
#imports
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport
import copy

from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation

from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)

from ema_workbench.analysis import plotting, plotting_util


In [49]:
#running the model through EMA workbench
ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=100, policies=3)

[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments

  0%|                                                  | 0/300 [00:00<?, ?it/s][MainProcess/INFO] performing experiments sequentially

100%|████████████████████████████████████████| 300/300 [11:30<00:00,  2.30s/it]
[MainProcess/INFO] experiments finished


In [51]:
experiments, outcomes = results
print(outcomes.keys())


dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


In [32]:
outcomes.keys()

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])

In [33]:
outcomes_of_interest = ['A.4 Total Costs', 'A.4_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs']

In [52]:
fd = pd.DataFrame()
for outcome in outcomes_of_interest:
    fd[outcome] = outcomes[outcome]

In [53]:
fd

,A.4 Total Costs,A.4_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,3.904215e+07,0.0,1.420800e+09,0.000000
1,3.904215e+07,0.0,1.420800e+09,192.140197
2,3.904215e+07,0.0,1.420800e+09,6934.797511
3,3.904215e+07,0.0,1.420800e+09,473.963484
4,3.904215e+07,0.0,1.420800e+09,0.000000
...,...,...,...,...
295,4.676636e+07,0.0,5.046000e+08,5243.921137
296,4.676636e+07,0.0,5.046000e+08,44.812268
297,4.676636e+07,0.0,5.046000e+08,5153.839383
298,4.676636e+07,0.0,5.046000e+08,6171.747696


In [54]:
for columns in experiments.columns:
    fd[columns] = experiments[columns]

In [41]:
len(fd.columns)

57

In [55]:
fd

,A.4 Total Costs,A.4_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,3.904215e+07,0.0,1.420800e+09,0.000000,128,81.691798,10,0.787504,96.830738,1.5,...,9,3,1,5,9,5,2,126,123,dikesnet
1,3.904215e+07,0.0,1.420800e+09,192.140197,54,40.255142,1.0,0.346470,64.738154,1.5,...,9,3,1,5,9,5,2,127,123,dikesnet
2,3.904215e+07,0.0,1.420800e+09,6934.797511,124,199.002302,10,0.570466,157.411164,10,...,9,3,1,5,9,5,2,128,123,dikesnet
3,3.904215e+07,0.0,1.420800e+09,473.963484,8,311.539862,1.5,0.009909,59.972324,1.0,...,9,3,1,5,9,5,2,129,123,dikesnet
4,3.904215e+07,0.0,1.420800e+09,0.000000,116,269.556133,1.5,0.193439,36.175818,1.5,...,9,3,1,5,9,5,2,130,123,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4.676636e+07,0.0,5.046000e+08,5243.921137,86,165.494861,1.0,0.106758,250.928021,1.0,...,6,5,7,3,3,10,4,221,125,dikesnet
296,4.676636e+07,0.0,5.046000e+08,44.812268,98,325.962355,1.5,0.585723,38.445583,1.0,...,6,5,7,3,3,10,4,222,125,dikesnet
297,4.676636e+07,0.0,5.046000e+08,5153.839383,62,54.879802,1.5,0.136836,303.207044,10,...,6,5,7,3,3,10,4,223,125,dikesnet
298,4.676636e+07,0.0,5.046000e+08,6171.747696,100,285.410350,1.0,0.482068,314.161521,1.5,...,6,5,7,3,3,10,4,224,125,dikesnet


In [45]:
import matplotlib.pyplot as plt
import pandas as pd

from ema_workbench.analysis import prim
from ema_workbench.util import ema_logging

x = fd.iloc[:, 23:53]
y = fd.iloc[:, 0].values

prim_alg = prim.Prim(x, y, threshold=0.8, peel_alpha=0.1)
box1 = prim_alg.find_box()

box1.show_tradeoff()
print(box1.resample(21))
box1.inspect(21)
box1.inspect(21, style="graph")
box1.show_pairs_scatter(21)

plt.show()

AssertionError: 

In [57]:
fd

,A.4 Total Costs,A.4_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,3.904215e+07,0.0,1.420800e+09,0.000000,128,81.691798,10,0.787504,96.830738,1.5,...,9,3,1,5,9,5,2,126,0,dikesnet
1,3.904215e+07,0.0,1.420800e+09,192.140197,54,40.255142,1.0,0.346470,64.738154,1.5,...,9,3,1,5,9,5,2,127,0,dikesnet
2,3.904215e+07,0.0,1.420800e+09,6934.797511,124,199.002302,10,0.570466,157.411164,10,...,9,3,1,5,9,5,2,128,0,dikesnet
3,3.904215e+07,0.0,1.420800e+09,473.963484,8,311.539862,1.5,0.009909,59.972324,1.0,...,9,3,1,5,9,5,2,129,0,dikesnet
4,3.904215e+07,0.0,1.420800e+09,0.000000,116,269.556133,1.5,0.193439,36.175818,1.5,...,9,3,1,5,9,5,2,130,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4.676636e+07,0.0,5.046000e+08,5243.921137,86,165.494861,1.0,0.106758,250.928021,1.0,...,6,5,7,3,3,10,4,221,2,dikesnet
296,4.676636e+07,0.0,5.046000e+08,44.812268,98,325.962355,1.5,0.585723,38.445583,1.0,...,6,5,7,3,3,10,4,222,2,dikesnet
297,4.676636e+07,0.0,5.046000e+08,5153.839383,62,54.879802,1.5,0.136836,303.207044,10,...,6,5,7,3,3,10,4,223,2,dikesnet
298,4.676636e+07,0.0,5.046000e+08,6171.747696,100,285.410350,1.0,0.482068,314.161521,1.5,...,6,5,7,3,3,10,4,224,2,dikesnet


In [56]:
policies = experiments["policy"]
for i, policy in enumerate(np.unique(policies)):
    experiments.loc[policies == policy, "policy"] = str(i)

fd["policy"] = policies

KeyError: 'policy'

Error in callback <function flush_figures at 0x000001EFB5AC5670> (for post_execute):


KeyboardInterrupt: 

In [ ]:
sns.pairplot(fd, hue="policy", vars=list(fd.columns))
plt.show()

 16%|██████▏                                | 48/300 [23:02<2:00:58, 28.80s/it]


KeyboardInterrupt: 